In [1]:
from grid_world import negative_grid, standard_grid
from iterative_policy_evaluation import print_values, print_policy 
import numpy as np 
import operator 

In [2]:
gamma = 0.9
alpha = 0.1
possible_actions = ('U', 'D', 'L', 'R')

def max_dict(dictionary): 
    '''
    Returns the argmax key and the corresponding value 
    '''
    return max(dictionary.items(), key=operator.itemgetter(1))

def random_action(a, epsilon = 0.1): 
    '''
    Epsilon greedy policy 
    '''
    p = np.random.random()
    return a if p < (1 - epsilon) else np.random.choice(possible_actions)

In [3]:
# initialize grid 
grid = negative_grid(step_cost=-0.1)

# initialize Q 
Q = {}
for state in grid.all_states(): 
    Q[state] = {}
    for action in possible_actions: 
        Q[state][action] = 0
        
# let's also keep track of how many times Q[s] has been updated
update_counts = {}
update_counts_sa = {}

for s in grid.all_states():
    update_counts_sa[s] = {}
    for a in possible_actions:
        update_counts_sa[s][a] = 1.0

In [4]:
# the Q-learning algorithm

'''
Q-learning is very similar to SARSA however. Using Q-learning update we update Q(s, a) w.r.t max(Q(s', a')). 
However, we don't neccessarily pick that action due to the randomness of an epsilon-greedy policy. 
Whereas in SARSA we updated Q(s, a) based on the action a' that was the result from using an epsilon-greedy policy.
'''

NUM_EPISODES = 10000
t = 1

for ep in range(NUM_EPISODES): 
    if ep % 100 == 0: 
        t += 0.02 
        
    # initialize starting state 
    state = (2, 0)
    grid.set_state(state)
    
    # pick action 
    action, _ = max_dict(Q[s])
    
    while not grid.game_over(): 
        # make action 
        action = random_action(action, epsilon = 0.5/t)
        
        reward = grid.move(action)
        
        # store s' and a' 
        state_prime = grid.current_state()
        action_prime, _ = max_dict(Q[state_prime])
        
        # for the SARSA algorithm this section is uncommented. 
        # action_prime = random_action(action_prime, epsilon = 0.5/t)
        
        alph = alpha / update_counts_sa[state][action]
        Q[state][action] += alph * (reward + gamma * Q[state_prime][action_prime] - Q[state][action])
        
        update_counts[state] = update_counts.get(state, 0) + 1
        update_counts_sa[state][action] += 0.005
        
        state, action = state_prime, action_prime 
        
        
policy = {}
V = {}
for state in grid.actions.keys(): 
    policy[state], V[state] = max_dict(Q[state])
    
    
print("values:")
print_values(V, grid)
print("policy:")
print_policy(policy, grid)

values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
